### School Dropout - Tensor ML

### Import Relevant Libraries

In [21]:
import numpy as np
import tensorflow as tf

### Data

In [23]:
# let's create a temporary variable npz to store the three data
npz = np.load('school_dropout_data_train.npz', allow_pickle=True)

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('school_dropout_data_validation.npz', allow_pickle=True)
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('school_dropout_data_test.npz', allow_pickle=True)
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

### Model

In [26]:
input_size = 30
output_size = 2

hidden_layer_size = 100

#define how the model will look like
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

# Choose optimizer and loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# Set the batch size
batch_size = 100

#Set a maximum number of training epochs
max_epochs = 100

#Set an early stopping mechanism
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
model.fit(
    train_inputs,
    train_targets,
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping],
    validation_data=(validation_inputs, validation_targets),
    verbose=2
    
)

Epoch 1/100
22/22 - 1s - 23ms/step - accuracy: 0.7677 - loss: 0.5260 - val_accuracy: 0.8526 - val_loss: 0.3813
Epoch 2/100
22/22 - 0s - 2ms/step - accuracy: 0.8421 - loss: 0.3667 - val_accuracy: 0.8691 - val_loss: 0.3307
Epoch 3/100
22/22 - 0s - 2ms/step - accuracy: 0.8609 - loss: 0.3305 - val_accuracy: 0.8788 - val_loss: 0.3179
Epoch 4/100
22/22 - 0s - 2ms/step - accuracy: 0.8659 - loss: 0.3110 - val_accuracy: 0.8581 - val_loss: 0.3260
Epoch 5/100
22/22 - 0s - 2ms/step - accuracy: 0.8719 - loss: 0.2984 - val_accuracy: 0.8747 - val_loss: 0.3121
Epoch 6/100
22/22 - 0s - 2ms/step - accuracy: 0.8779 - loss: 0.2887 - val_accuracy: 0.8802 - val_loss: 0.3130
Epoch 7/100
22/22 - 0s - 2ms/step - accuracy: 0.8926 - loss: 0.2761 - val_accuracy: 0.8719 - val_loss: 0.3170


### Test the model

In [29]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - accuracy: 0.8838 - loss: 0.2984


In [31]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.30. Test accuracy: 88.84%
